In [1]:
from collections import defaultdict
import json
import re

from bs4 import BeautifulSoup
import pandas as pd
from requests import Session


In [2]:
s = Session()
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                         'AppleWebKit/537.36 (KHTML, like Gecko) '\
                         'Chrome/75.0.3770.80 Safari/537.36',
          'Content-Type': 'application/json;charset=UTF-8'}
# Add headers
s.headers.update(headers)

In [3]:
URL = 'https://www.autouncle.dk/en/cars_search?page=100'

In [4]:
r = s.get(URL)

In [5]:
r.ok

True

In [6]:
r

<Response [200]>

In [7]:
soup = BeautifulSoup(r.content,'lxml')

In [11]:
# div class='data', get data-featured-caousels --> list of dict
response_data = soup.find('div',{'data-featured-carousels':re.compile('.*')})

In [12]:
data = json.loads(response_data['data-featured-carousels'])

In [13]:
pd.DataFrame(data[0]['data']['cars']).head()

,au_rating,co2_emission,distance_to_car,energy_consumption,formatted_price,fuel_economy,headline,id,image_url,is_featured,...,link_target_blank,location_complete,missing_image_fallback,price,price_difference_formatted,rating_label,redirect_url,source_name,valuation_label,valuation_value
0,3,None,None,None,"DKK 464,000",17.5 Km/l,"VW Tiguan 2,0 TDI SCR Highline 4Motion DSG 190...",2857462,https://images.autouncle.com/da/car_images/med...,True,...,True,"4760 Vordingborg, Sjælland",False,464000,"Save DKK 10,000",Fair price,/en/outgoing_link/carsten-jensen-biler-dk/2857...,Carsten-jensen-biler.dk,Valuation,"DKK 474,000"
1,3,None,None,None,"DKK 459,900",21.3 Km/l,"BMW X2 2,0 xDrive20d aut.",2857862,https://images.autouncle.com/da/car_images/med...,True,...,True,"8210 Aarhus, Midtjylland",False,459900,"Save DKK 1,900",Fair price,/en/outgoing_link/bayern-autogroup-aarhus/2857...,Bayern AutoGroup Aarhus,Valuation,"DKK 458,000"
2,3,None,None,None,"DKK 459,900",16.4 Km/l,"BMW X2 2,0 xDrive20i aut.",2887980,https://images.autouncle.com/da/car_images/med...,True,...,True,"8210 Aarhus, Midtjylland",False,459900,"Save DKK 5,900",Fair price,/en/outgoing_link/bayern-autogroup-aarhus/2887...,Bayern AutoGroup Aarhus,Valuation,"DKK 454,000"
3,4,None,None,None,"DKK 459,900",22.7 Km/l,"Mercedes C220 d 2,0 AMG Line aut.",2885820,https://images.autouncle.com/da/car_images/med...,True,...,True,"7100 Vejle, Syddanmark",False,459900,"Save DKK 27,100",Good price,/en/outgoing_link/starmark-vejle/2885820/50605...,Starmark Vejle,Valuation,"DKK 487,000"
4,4,None,None,None,"DKK 459,900",22.7 Km/l,"Mercedes C220 d 2,0 AMG Line aut.",2892785,https://images.autouncle.com/da/car_images/med...,True,...,True,"7100 Vejle, Syddanmark",False,459900,"Save DKK 30,100",Good price,/en/outgoing_link/starmark-vejle/2892785/50692...,Starmark Vejle,Valuation,"DKK 490,000"


In [14]:
items = soup.find_all('div',class_='listing-item')

In [15]:
items[0]

<div class="listing-item" data-car-id="2840724" data-preload-fb-comments="false" itemscope="" itemtype="http://schema.org/Product">
<div class="listing-item-container">
<div class="listing-item-car-information">
<div class="car-picture">
<span class="car-name-label"><span class="car-freshness-label car-freshness-label--reduced">-16%</span></span>
<meta content="https://images.autouncle.com/da/car_images/1266ca84-e63d-442a-8fb8-b707e3034fdd_hyundai-i30-1-6-crdi-comfort-90hk-5d.jpg" itemprop="image"/>
<span class="picture-count-indicator">
1/4
</span>
<img alt="used Hyundai i30 1,6 CRDi Comfort 90HK 5d" data-image-url="https://images.autouncle.com/da/car_images/medium_1266ca84-e63d-442a-8fb8-b707e3034fdd_hyundai-i30-1-6-crdi-comfort-90hk-5d.jpg" src="data:image/gif;base64,R0lGODdhAQABAPAAAP///wAAACwAAAAAAQABAEACAkQBADs="/><noscript><img alt="used Hyundai i30 1,6 CRDi Comfort 90HK 5d" lazyload="true" src="https://images.autouncle.com/da/car_images/medium_1266ca84-e63d-442a-8fb8-b707e3034f

In [16]:
store = defaultdict(list)

In [17]:
for item in items:
    store['car'].append(item.find('img')['alt'])
    store['cost'].append(item.find('span', class_='price')['content'])
    store['should_cost'].append(item.find('span', class_='should-cost').find_all('span')[-1].get_text(strip=True))
    [store[i['class'][-1]].append(i.span.get_text(strip=True))  for i in item.find_all('li')]
     
    

In [18]:
pd.DataFrame(store)

,car,cost,should_cost,year,km,engine,location
0,"used Hyundai i30 1,6 CRDi Comfort 90HK 5d",25000,"DKK 31,000",2008,"246,000km",1.6 Diesel (90 hp) 21.3 Km/l,"Dealer, 4690, Sjælland"
1,"used Dacia Logan 0,9 TCe 90 Laureate MCV",79900,"DKK 90,000",2014,"92,000km",0.9 Petrol (90 hp) 20.0 Km/l,"Dealer, 8930 Randers NØ, Midtjylland"
2,"used Mazda 6 2,0 Touring 147HK Stc 6g",22000,"DKK 31,000",2005,"220,000km",2.0 Petrol (147 hp) 12.5 Km/l,"Private seller, 8670, Midtjylland"
3,"used Skoda Octavia 1,2 TSI Family 105HK 5d",104900,"DKK 120,000",2012,"87,000km",1.2 Petrol (105 hp) 17.5 Km/l,"Dealer, 6500, Syddanmark"
4,"used Kia cee'd SW 1,6 CRDI Style Plus Clim 136...",164900,"DKK 176,000",2016,"47,000km",1.6 Diesel (136 hp) 25.6 Km/l,"Dealer, 8960, Midtjylland"
5,"used Ford Ka 1,2 Trend Plus 69HK 3d",34800,"DKK 41,000",2011,"68,000km",1.2 Petrol (69 hp) 19.6 Km/l,"Dealer, 4760, Sjælland"
6,"used Ford S-MAX 2,0 TDCi Titanium 180HK 6g - P...",329900,"DKK 352,000",2016,"11,000km",2.0 Diesel (180 hp) 20.0 Km/l,"Dealer, 7160 Tørring, Midtjylland"
7,"used Mitsubishi ASX 1,6 Invite 2WD 117HK 5d",139900,"DKK 151,000",2015,"63,000km",1.6 Petrol (117 hp) 17.2 Km/l,"Dealer, 2605, Hovedstaden"
8,"used Toyota Avensis 1,8 SD MAN.",47600,"DKK 59,000",2006,"146,000km",1.8 Petrol (129 hp) 13.9 Km/l,"Private seller, 5250 Odense SV, Syddanmark"
9,,90000,"DKK 100,000",2011,"213,500km",1.6 Diesel (105 hp) 22.7 Km/l,"Private seller, 6780 Skærbæk, Syddanmark"


In [ ]:
#How to stop
soup.find('div',{'id':'pagination'}).find_all('span')[-1]['class']